In [1]:
%%capture
!pip install mljar-supervised;
!pip install gdown
!pip install imblearn



In [3]:

# Download both interpro and rast annotations for genomes for this to work. Stored in data/ Both files are in .gitignore
import os



# Check and download allgenomes.RAST.txt.ps.tsv file
if not os.path.exists("data/rast_features.tsv"):
    print("Downloading File data/rast_features.tsv")
    !gdown --no-check-certificate 1dDLqkjN0YGKa7mNDKhTmGSeZZWt1T3cl -O data/rast_features.tsv
else:
    print("File data/rast_features.tsv already exists")

# Check and download allgenomes.interpro.txt.ps.tsv file
if not os.path.exists("data/interpro_features.tsv"):
    print("Downloading File data/interpro_features.tsv")
    !gdown --no-check-certificate 13I7lBj02KEsA4OowW89h0zW8tLsmQrrb -O data/interpro_features.tsv
else:
    print("File data/interpro_features.tsv already exists")
    
    

File data/rast_features.tsv already exists
File data/interpro_features.tsv already exists


In [3]:
import uuid
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.metrics import classification_report, accuracy_score
import shap
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from IPython.display import display, HTML
import joblib



def get_X_y(feature_df, pheno_full_df, phenotype_name, variance_threshold):

   
    # Read phenotype data
    phenotype_df = pheno_full_df[phenotype].dropna()

    # Merge genotype and phenotype df 
    merged_df = feature_df.merge(phenotype_df, left_index=True, right_index=True, how='inner')

    # Remove all features small variance as they don't have much discriminatory power
    constant_filter = VarianceThreshold(threshold=variance_threshold)
    constant_filter.fit(merged_df)
    selected_columns_indices = constant_filter.get_support()
    selected_column_names = merged_df.columns[selected_columns_indices]
    df_filtered = merged_df[selected_column_names]


    # Get X and Y
    X = df_filtered.drop(columns=[phenotype])
    y = df_filtered[phenotype]
    
    return [X,y]


def generate_html_table(df: pd.DataFrame):
        """Display a pandas.DataFrame as jQuery DataTables"""

        # Generate random container name
        id_container = uuid.uuid1()
        output = """
    <div id="datatable-container-{id_container}">
      <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.7.0/jquery.min.js"></script>
      <script type="text/javascript" src="https://cdn.datatables.net/1.13.5/js/jquery.dataTables.min.js"></script>
      <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.13.5/css/jquery.dataTables.min.css"/>
      <script type="text/javascript">
        $(document).ready( function () {{
            $('#BGCtable').DataTable();
        }});
      </script>
      <!-- Insert table below -->
      {table}
    </div>
        """.format(
            id_container=id_container,
            table=df.to_html(
                index=False,
                table_id="classification",
                classes="display"
            ),
        )
        return output




def get_shap_plot(model, X_test, X, title, result_dir):
    # Initialize the SHAP explainer
    explainer = shap.TreeExplainer(model)

        # Calculate SHAP values for the test set
    shap_values = explainer.shap_values(X_test)

        # Get feature importance based on SHAP values
    mean_abs_shap = np.mean(np.abs(shap_values[1]), axis=0)
    feature_importance = pd.DataFrame({'feature': X.columns, 'importance': mean_abs_shap})
    feature_importance = feature_importance.sort_values(by='importance', ascending=False)

    # Select the top 100 features 
    top_features = feature_importance.head(20)[['feature', 'importance']]
    
    tsv_file =  result_dir + "/" + title + ".tsv"
    # Save the top_features DataFrame to a TSV file
    top_features.to_csv(tsv_file , sep='\t', index=False)

    # Concatenate the feature and importance columns into a single string separated by '#'
    top_features['combined'] = top_features.apply(lambda row: f"{row['feature']} // {row['importance']}", axis=1)

     # Extract the 'combined' column as a list of strings
    concatenated_features = top_features['combined'].tolist()

     # If you want a single string with all features separated by a space or newline:
    formatted_top_features = ' # '.join(concatenated_features)
  


    plt.switch_backend('Agg')
    
    # Plot SHAP summary plot for the top 20 features
    plt.figure(figsize=(20, 10))  # Adjust width (20) and height (10) as desired
    plt.title(title)
    shap.summary_plot(shap_values[1], X_test, max_display=20, plot_size=None)
    img_path = result_dir + "/" + title + ".png"
    plt.savefig(img_path)
    plt.close()

    return [formatted_top_features, img_path]



def get_shap_plot_catboost(model, X_test, X, title, result_dir):
    # Create explainer object
    explainer = shap.Explainer(model)
    
    # Calculate SHAP values for all instances in your dataset
    shap_values = explainer.shap_values(X)

    # Get feature importance based on SHAP values
    mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
    feature_importance = pd.DataFrame({'feature': X.columns, 'importance': mean_abs_shap})
    feature_importance = feature_importance.sort_values(by='importance', ascending=False)

    # Select the top 100 features 
    top_features = feature_importance.head(20)[['feature', 'importance']]
    
    tsv_file =  result_dir + "/" + title + ".tsv"
    # Save the top_features DataFrame to a TSV file
    top_features.to_csv(tsv_file , sep='\t', index=False)
    
    # Concatenate the feature and importance columns into a single string separated by '#'
    top_features['combined'] = top_features.apply(lambda row: f"{row['feature']} // {row['importance'] }", axis=1)

     # Extract the 'combined' column as a list of strings
    concatenated_features = top_features['combined'].tolist()

     # If you want a single string with all features separated by a space or newline:
    formatted_top_features = ' # '.join(concatenated_features)
  


    plt.switch_backend('Agg')
    plt.figure(figsize=(20, 10))  # Adjust width (20) and height (10) as desired
    plt.title(title)
    shap.summary_plot(shap_values, X, max_display=20, plot_size=None)
    img_path = result_dir + "/" + title + ".png"
    plt.savefig(img_path)
    plt.close()

    return formatted_top_features, img_path

    
    
def get_classifier_report(X, y, phenotype_name, result_dir):

    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    import xgboost as xgb
    import lightgbm as lgb
    import catboost as cb
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, confusion_matrix
    from sklearn.feature_selection import SelectKBest, chi2
    import pandas as pd



    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1176)
    
    # Compute the class ratio
    minority_class_count = sum(y_train == 1)  # Assuming '1' denotes the minority class
    majority_class_count = len(y_train) - minority_class_count
    class_ratio = minority_class_count / majority_class_count

    # If the class ratio is too low (e.g., below 0.75), perform over-sampling
    if class_ratio < 0.75:
        ros = RandomOverSampler(sampling_strategy=0.75, random_state=1176)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
    else:
       # If not, simply keep the original samples
        X_train_resampled, y_train_resampled = X_train, y_train

    # If the class ratio after over-sampling exceeds a threshold (e.g., greater than 0.9), perform under-sampling
    if class_ratio > 0.9:
        rus = RandomUnderSampler(sampling_strategy=1.0, random_state=1176)
        X_train_resampled, y_train_resampled = rus.fit_resample(X_train_resampled, y_train_resampled)


    # Dictionary to store the classifiers and their parameters
    classifiers = {
        'Decision Tree': {
            'model': DecisionTreeClassifier(),
            'params': {}
        },
        'Random Forest': {
            'model': RandomForestClassifier(),
            'params': {'n_estimators': 1000, 'max_depth': None, 'random_state': 100}
        }, 

        'CatBoost': {
            'model': cb.CatBoostClassifier(),
            'params': {'iterations': 1000, 'depth': 6, 'learning_rate': 0.1, 'random_state': 42, 'verbose': False}
        }

    }

    html_table_rows = []

    img_paths = []

    # Train and evaluate each classifier
    for clf_name, clf_data in classifiers.items():
        print ("Running " + clf_name + " for phenotype " + phenotype_name)
        model = clf_data['model']
        params = clf_data['params']

        # Train the classifier
        model.set_params(**params)
        model.fit(X_train_resampled, y_train_resampled)

        # Make predictions on the test set
        y_pred = model.predict(X_test)



        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        
        confusion_matrix_info = confusion_matrix(y_test, y_pred)
        
        # Extracting values
        TP = confusion_matrix_info[0, 0]
        FP = confusion_matrix_info[0, 1]
        FN = confusion_matrix_info[1, 0]
        TN = confusion_matrix_info[1, 1]
        
        confusion_matrix_data = "TP=" + str(TP) + " TN=" + str(TN) + " FP=" + str(FP) + "FN=" + str(FN)
   
        title = clf_name + "___" + phenotype_name
        
        # Print shap
        if clf_name == "CatBoost":
            top_features, img_path = get_shap_plot_catboost(model, X_test, X, title, result_dir)
            img_paths.append(img_path)
        else:
            top_features, img_path = get_shap_plot (model, X_test, X, title, result_dir)
            img_paths.append(img_path)

        # Append row to the HTML table
        html_table_rows.append([clf_name, accuracy, precision, recall, f1, balanced_accuracy, confusion_matrix_data, top_features])
        
        

            
            

    # Create a DataFrame to display the results
    headers = ["Model", "Accuracy", "Precision", "Recall", "F1-score", "Balanced Accuracy", "Confusion Matrix", "Top features"]
    report_df = pd.DataFrame(html_table_rows, columns=headers)
    fname = result_dir + "/" + phenotype_name + "_complete.tsv"
    report_df.to_csv(fname , sep='\t', index=False)
    report_html_content = generate_html_table (report_df)
    print ("######### Combined report for " + phenotype_name + "#####################")
    display(HTML(report_html_content))
    html_img = ""
    for img_path in img_paths:
        html_img += "<img src='" + img_path + "'</></br>"
        
    display(HTML(html_img))

    



In [4]:

result_dir = "ml_results/non_metabolic_phenotypes"

VARIANCE_THRESHOLD = 0.0001

# Read rast features
rast_annotated_df = pd.read_csv("data/rast_features.tsv", sep = "\t", index_col=0)
# Print some stats about data
# Cleaning function
#def clean_column_name(col_name):
#    return ''.join(filter(str.isalnum, col_name.replace("[", "").replace("]", "").replace("<", "")))

# Cleaning function
def clean_column_name(col_name):
    for char in [",", "[", "]", "<"]:
        col_name = col_name.replace(char, "")
    return col_name


# Apply the cleaning function to each column name
rast_annotated_df.columns = [clean_column_name(col) for col in rast_annotated_df.columns]




metabolic_phenotypes_df = pd.read_csv("data/non_metabolic_phenotypes_bacdive.tsv", sep = "\t", index_col=0)  


for phenotype in metabolic_phenotypes_df:
    # All phenotype names have -- in them
    if "strain" in phenotype:
        continue
    

    try:
        X, y = get_X_y(rast_annotated_df, metabolic_phenotypes_df, phenotype,VARIANCE_THRESHOLD )

        num_genome, num_features = X.shape
        display(HTML("<H2> Phenotype: " + phenotype + "</h2>"))
        print ("Numer of genomes:" + str(num_genome) + "\n" + "Number of genomic features :" + str(num_features)) 
        print ("Shape of y:" + str(y.shape))
        count_1 = sum(y==1)
        count_0 = sum(y==0)
        print ("Count of 1: " + str(count_1) + "\n" + "Count of 0: " + str(count_0))
    
        get_classifier_report(X,y, phenotype, result_dir)
        

    except Exception as e:
        print (e)


Numer of genomes:3488
Number of genomic features :5745
Shape of y:(3488,)
Count of 1: 1259
Count of 0: 2229
Running Decision Tree for phenotype gram_positive


/Users/4pz/opt/anaconda3/lib/python3.8/site-packages/shap/plots/_beeswarm.py:927: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


Running Random Forest for phenotype gram_positive


/Users/4pz/opt/anaconda3/lib/python3.8/site-packages/shap/plots/_beeswarm.py:927: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


Running CatBoost for phenotype gram_positive


...

/Users/4pz/opt/anaconda3/lib/python3.8/site-packages/shap/plots/_beeswarm.py:927: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


Running CatBoost for phenotype spore_formation


/Users/4pz/opt/anaconda3/lib/python3.8/site-packages/shap/plots/_beeswarm.py:927: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


######### Combined report for spore_formation#####################


Model,Accuracy,Precision,Recall,F1-score,Balanced Accuracy,Confusion Matrix,Top features
Decision Tree,0.919922,0.920110,0.919922,0.920012,0.899907,TP=352 TN=119 FP=21FN=20,SSO:000025068__Sporulation protein // 0.16975429726264474 # SSO:000007697__Stage III sporulation protein AA // 0.08118373209128858 # SSO:000023921__Redox-sensing transcriptional repressor Rex // 0.0756220169745765 # SSO:000020544__Nitric oxide synthase oxygenase (EC 1.-.-.-) // 0.03979136913022205 # SSO:000002702__Ferredoxin (EC 1.18.1.3 ) // 0.031001852484789743 # SSO:000012811__DNA polymerase III subunit delta' // 0.02418650496698533 # SSO:000010833__Arylesterase // 0.022943764554313087 # SSO:000001864__Cysteine synthase (EC 2.5.1.47) // 0.019338336181067277 # SSO:000025027__Spore cortex-lytic enzyme // 0.011946749416486479 # SSO:000008961__Xanthine phosphoribosyltransferase (EC 2.4.2.22) // 0.009201137457320333 # SSO:000042567__mercury(II) reductase (EC 1.16.1.1) // 0.00790315893766282 # SSO:000017637__Haloalkane dehalogenase (EC 3.8.1.5) // 0.007233455939333527 # SSO:000009661__4-alpha-glucanotransferase // 0.0068755152676560245 # SSO:000005097__N-acetyl-gamma-glutamyl-phosphate reductase (EC 1.2.1.38) // 0.00675979253332924 # SSO:000010697__Anti-sigma-V factor RsiV // 0.005624400509020343 # SSO:000009313__stage V sporulation protein B // 0.005311482575733133 # SSO:000016824__Flavin reductase (EC 1.5.1.30) // 0.004341838029500944 # SSO:000013628__FAD:protein FMN transferase (EC 2.7.1.180) // 0.0038540995852703415 # SSO:000001044__Arginine decarboxylase (EC 4.1.1.19) // 0.0037963745395961108 # SSO:000033054__radical SAM protein // 0.003391363557865968
Random Forest,0.958984,0.959337,0.958984,0.959121,0.951540,TP=361 TN=130 FP=12FN=9,SSO:000025068__Sporulation protein // 0.008942308715967854 # SSO:000023921__Redox-sensing transcriptional repressor Rex // 0.007896822036279293 # SSO:000002241__Demethylmenaquinone methyltransferase (EC 2.1.1.163) // 0.0073033454177757785 # SSO:000002702__Ferredoxin (EC 1.18.1.3 ) // 0.0070768265743266145 # SSO:000035439__DUF402 domain-containing protein // 0.006453343981082653 # SSO:000007937__Tetraacyldisaccharide 4'-kinase (EC 2.7.1.130) // 0.005980371709978124 # SSO:000022969__Protoporphyrinogen oxidase // 0.005928286444193033 # SSO:000007697__Stage III sporulation protein AA // 0.005872044620440602 # SSO:000006816__Pyridoxine 5'-phosphate synthase (EC 2.6.99.2) // 0.005827720485923993 # SSO:000004460__Lipid-A-disaccharide synthase (EC 2.4.1.182) // 0.005551366542448786 # SSO:000036573__LPS-assembly protein LptD // 0.005548932024581187 # SSO:000001005__Anti-sigma F factor (EC 2.7.11.1) // 0.005540486454511106 # SSO:000036507__L-glutamate gamma-semialdehyde dehydrogenase (EC 1.2.1.88) // 0.005517931787517067 # SSO:000007707__Stage IV sporulation protein A // 0.005305227105699486 # SSO:000025017__Spore coat protein // 0.005173540460607951 # SSO:000008104__TonB-dependent receptor // 0.005157838578703942 # SSO:000022976__PspA/IM30 family protein // 0.004722049204454313 # SSO:000025054__Sporulation integral membrane protein YtvI // 0.004688770634178325 # SSO:000008604__UDP-3-O-3-hydroxymyristoyl glucosamine N-acyltransferase (EC 2.3.1.-) // 0.004676343764029396 # SSO:000007702__Stage III sporulation protein AF // 0.004628008545276964
CatBoost,0.964844,0.964844,0.964844,0.964844,0.955562,TP=364 TN=130 FP=9FN=9,SSO:000025068__Sporulation protein // 0.4023731807869705 # SSO:000034380__ABC-F type ribosomal protection protein // 0.3570154517434232 # SSO:000007097__Ribonuclease Z (EC 3.1.26.11) // 0.35137754329920745 # SSO:000001091__Aspartate 1-decarboxylase (EC 4.1.1.11) // 0.24247620366035555 # SSO:000025053__Sporulation integral membrane protein YlbJ // 0.23666413294493455 # SSO:000035439__DUF402 domain-containing protein // 0.22684245218040083 # SSO:000023921__Redox-sensing transcriptional repressor Rex // 0.2242525353048122 # SSO:000042897__acetoacetate---CoA ligase (EC 6.2.1.16) // 0.22331510938567947 # SSO:000012992__DUF350 